# Monte Carlo assessment

A Monte Carlo simulation is a way to assess uncertainties and robustness given probabilities on the model inputs. 

Here we illustrate how it works with a design Monte Carlo. It is also called robust design because we can  evaluate the impact on the designed characteristics (here the `exchanger.surface`).

In [1]:
from cosapp.drivers import MonteCarlo, NonLinearSolver, RunSingleCase
from cosapp.recorders import DataFrameRecorder
from cosapp.utils.distributions import Normal

from cpu.systems import CPUSystem

In [2]:
cpu = CPUSystem("cpu")

In [3]:
mc = cpu.add_driver(MonteCarlo("mc"))

design = mc.add_child(NonLinearSolver("design"))
runner = design.add_driver(RunSingleCase("runner"))
design.extend(cpu.design_methods["exchanger_surface"])
runner.set_values({"T_cpu": 80.0, "cpu.usage": 100.0})

mc.add_recorder(DataFrameRecorder(includes=["*"]))
mc.draws = 100

# parameters for uncertainties in the data
T_air = cpu.fan.inwards.get_details("T_air")
# Set the distribution around the current value
T_air.distribution = Normal(best=1, worst=-0.5)
mc.add_random_variable("fan.T_air")
mc.add_response(["fan.tension", "cpu.usage", "T_cpu"])

In [4]:
cpu.run_drivers()
mc.recorder.export_data()

,Section,Status,Error code,Reference,T_cpu,controler.T_cpu,controler.high_threshold,controler.low_tension,controler.low_threshold,controler.max_tension,...,exchanger.heat_flow,exchanger.max_mass_flow,exchanger.surface,fan.T_air,fan.design_tension,fan.fl_out.T,fan.fl_out.mass_flow,fan.mass_flow_max,fan.mass_flow_scalar,fan.tension
0,,,0,0,80.0,80.0,60.0,0.0,40.0,12.0,...,105.0,1.0,0.008521,40.250000,12.0,40.250000,1.0,1.0,1.0,12.0
1,,,0,1,80.0,80.0,60.0,0.0,40.0,12.0,...,105.0,1.0,0.008627,40.738085,12.0,40.738085,1.0,1.0,1.0,12.0
2,,,0,2,80.0,80.0,60.0,0.0,40.0,12.0,...,105.0,1.0,0.008418,39.761915,12.0,39.761915,1.0,1.0,1.0,12.0
3,,,0,3,80.0,80.0,60.0,0.0,40.0,12.0,...,105.0,1.0,0.008472,40.019421,12.0,40.019421,1.0,1.0,1.0,12.0
4,,,0,4,80.0,80.0,60.0,0.0,40.0,12.0,...,105.0,1.0,0.008703,41.082434,12.0,41.082434,1.0,1.0,1.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,,,0,95,80.0,80.0,60.0,0.0,40.0,12.0,...,105.0,1.0,0.008256,38.975193,12.0,38.975193,1.0,1.0,1.0,12.0
96,,,0,96,80.0,80.0,60.0,0.0,40.0,12.0,...,105.0,1.0,0.008536,40.320968,12.0,40.320968,1.0,1.0,1.0,12.0
97,,,0,97,80.0,80.0,60.0,0.0,40.0,12.0,...,105.0,1.0,0.008647,40.831204,12.0,40.831204,1.0,1.0,1.0,12.0
98,,,0,98,80.0,80.0,60.0,0.0,40.0,12.0,...,105.0,1.0,0.008436,39.847568,12.0,39.847568,1.0,1.0,1.0,12.0
